<a href="https://colab.research.google.com/github/suyashchaturvedi26/Fama-French-Model/blob/main/Fama_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xlsxwriter

In [ ]:
# Importing the libraries
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pylab import rcParams
from scipy.stats import f_oneway
from scipy.stats import ttest_ind as T
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pandas_datareader import DataReader as web
from sklearn import linear_model
import xlsxwriter

In [ ]:
%matplotlib inline
np.random.seed(8)
warnings.filterwarnings('ignore')
rcParams['figure.figsize']=20,10
rcParams['font.size']=30

## CAR Calculation

In [ ]:
# Reading all the data from excel sheets 
Historical_Data=pd.read_excel('NMP Output Sheet.xlsx',sheet_name='Historical Stock Prices')
Three_Day_CAR =pd.read_excel('NMP Output Sheet.xlsx',sheet_name='3 Day CAR')
Five_Day_CAR =pd.read_excel('NMP Output Sheet.xlsx',sheet_name='5 Day CAR')
Seven_Day_CAR =pd.read_excel('NMP Output Sheet.xlsx',sheet_name='7 Day CAR')
df_factors=pd.read_excel('NMP Output Sheet.xlsx',sheet_name='Factors')

In [ ]:
Historical_Data=Historical_Data.merge(df_factors,on='Date',how='left')
Historical_Data.head()

In [ ]:
Company_List=Historical_Data['Symbol'].unique()
Beta_Coef = pd.DataFrame(columns = ['Symbol', 'x1', 'x2','x3','x4','Intercept'])
tmp=pd.DataFrame(columns = ['Symbol', 'x1', 'x2','x3','x4','Intercept'])

for l in Company_List:
  df_filtered=Historical_Data[Historical_Data['Symbol']==l] 
  y=df_filtered['Returns_Close']
  X=df_filtered[['Rf %','Rm-Rf %','SMB %','HML %']]
  regr = linear_model.LinearRegression()
  regr.fit(X, y)
  coef=pd.DataFrame(regr.coef_)
  # print(regr.coef_)
  tmp[['x1','x2','x3','x4']]=coef.T
  tmp['Intercept']=regr.intercept_
  tmp['Symbol']=l
  Beta_Coef=Beta_Coef.append(tmp,True)


Beta_Coef.head()

In [ ]:
def data_merge(df1,df2,df3):
  df1=df1.merge(df2,on='Date',how='left')
  df1=df1.merge(df3,on="Symbol",how='left')
  return df1

Three_Day_CAR=data_merge(Three_Day_CAR,df_factors,Beta_Coef)
Five_Day_CAR=data_merge(Five_Day_CAR,df_factors,Beta_Coef)
Seven_Day_CAR=data_merge(Seven_Day_CAR,df_factors,Beta_Coef)

In [ ]:
def predicted_return(df):
  df['Predicted Return']=df['x1']*df['Rf %']+df['x2']*df['Rm-Rf %']+df['x3']*df['SMB %']+df['x4']*df['HML %'] + df['Intercept']
  return df


Three_Day_CAR=predicted_return(Three_Day_CAR)

Five_Day_CAR=predicted_return(Five_Day_CAR)

Seven_Day_CAR=predicted_return(Seven_Day_CAR)

In [ ]:
def abnormal_return(df):
  df=df[['Date','Symbol','Returns_Close','Predicted Return']]
  df['Abnormal Return']=df['Returns_Close'] - df['Predicted Return']
  df['Date']=pd.to_datetime(df['Date'])
  df['Year']=df['Date'].dt.year

  return df

Three_Day_CAR=abnormal_return(Three_Day_CAR)

Five_Day_CAR=abnormal_return(Five_Day_CAR)

Seven_Day_CAR=abnormal_return(Seven_Day_CAR)

## **Demonetization Analysis**

In [ ]:
d=datetime.datetime(2016, 11, 8)

### 3 Day CAR
Three_Day_Post_Demo=Three_Day_CAR.loc[Three_Day_CAR.Year > 2016]
Three_Day_Pre_Demo=Three_Day_CAR.loc[Three_Day_CAR.Date < d]

Three_Day_Pre_Demo.drop(columns=['Year'],inplace=True)
Three_Day_Post_Demo.drop(columns=['Year'],inplace=True)

### 5 Day CAR
Five_Day_Post_Demo=Five_Day_CAR.loc[Five_Day_CAR.Year > 2016]
Five_Day_Pre_Demo=Five_Day_CAR.loc[Five_Day_CAR.Date < d]

Five_Day_Pre_Demo.drop(columns=['Year'],inplace=True)
Five_Day_Post_Demo.drop(columns=['Year'],inplace=True)

### 7 Day CAR
Seven_Day_Post_Demo=Seven_Day_CAR.loc[Seven_Day_CAR.Year > 2016]
Seven_Day_Pre_Demo=Seven_Day_CAR.loc[Seven_Day_CAR.Date < d]

Seven_Day_Pre_Demo.drop(columns=['Year'],inplace=True)
Seven_Day_Post_Demo.drop(columns=['Year'],inplace=True)

In [ ]:
def CAR(df):
  df1=df.groupby(['Symbol']).sum()
  df1.rename(columns={"Abnormal Return": "CAR"},inplace=True)
  df1['AAR']=df.groupby(['Symbol']).mean()['Abnormal Return']
  df1.drop(columns=['Returns_Close','Predicted Return','AAR'],inplace=True)
  return df1


Three_Day_Pre_Demo_Data=CAR(Three_Day_Pre_Demo)
Three_Day_Post_Demo_Data=CAR(Three_Day_Post_Demo)


Five_Day_Pre_Demo_Data=CAR(Five_Day_Pre_Demo)
Five_Day_Post_Demo_Data=CAR(Five_Day_Post_Demo)

Seven_Day_Pre_Demo_Data=CAR(Seven_Day_Pre_Demo)
Seven_Day_Post_Demo_Data=CAR(Seven_Day_Post_Demo)

In [ ]:
t2, p2 = T(Three_Day_Pre_Demo_Data,Three_Day_Post_Demo_Data)
t3, p3 = T(Five_Day_Pre_Demo_Data,Five_Day_Post_Demo_Data)
t4, p4 = T(Seven_Day_Pre_Demo_Data,Seven_Day_Post_Demo_Data)


t={'3 Day CAR':t2[0],'5 Day CAR':t3[0],'7 Day CAR':t4[0]}
p={'3 Day CAR':p2[0],'5 Day CAR':p3[0],'7 Day CAR':p4[0]}

Demo_t_test = pd.DataFrame.from_dict(t,orient='index',columns=['T-Stat'])
Demo_p_test = pd.DataFrame.from_dict(p,orient='index',columns=['P-Stat'])
Demo_Stats=Demo_t_test.merge(Demo_p_test,left_index=True, right_index=True)
Demo_Stats

## **COVID - 19 Analysis**

In [ ]:
### 3 Day CAR
Three_Day_Post_COVID=Three_Day_CAR.loc[Three_Day_CAR.Year > 2019]
Three_Day_Pre_COVID=Three_Day_CAR.loc[Three_Day_CAR.Year < 2020]

Three_Day_Pre_COVID.drop(columns=['Year'],inplace=True)
Three_Day_Post_COVID.drop(columns=['Year'],inplace=True)

### 5 Day CAR
Five_Day_Post_COVID=Five_Day_CAR.loc[Five_Day_CAR.Year > 2019]
Five_Day_Pre_COVID=Five_Day_CAR.loc[Five_Day_CAR.Year < 2020]

Five_Day_Pre_COVID.drop(columns=['Year'],inplace=True)
Five_Day_Post_COVID.drop(columns=['Year'],inplace=True)

### 7 Day CAR
Seven_Day_Post_COVID=Seven_Day_CAR.loc[Seven_Day_CAR.Year > 2019]
Seven_Day_Pre_COVID=Seven_Day_CAR.loc[Seven_Day_CAR.Year < 2020]

Seven_Day_Pre_COVID.drop(columns=['Year'],inplace=True)
Seven_Day_Post_COVID.drop(columns=['Year'],inplace=True)

In [ ]:
def car_covid(df):
  df1=df.groupby(['Symbol']).sum()
  df1.rename(columns={"Abnormal Return": "CAR"},inplace=True)
  df1['AAR']=df.groupby(['Symbol']).mean()['Abnormal Return']
  df1.drop(columns=['Returns_Close','Predicted Return','AAR'],inplace=True)
  
  return df1

Three_Day_Pre_COVID_Data=car_covid(Three_Day_Pre_COVID)
Three_Day_Post_COVID_Data=car_covid(Three_Day_Post_COVID)

Five_Day_Pre_COVID_Data=car_covid(Five_Day_Pre_COVID)
Five_Day_Post_COVID_Data=car_covid(Five_Day_Post_COVID)

Seven_Day_Pre_COVID_Data=car_covid(Seven_Day_Pre_COVID)
Seven_Day_Post_COVID_Data=car_covid(Seven_Day_Post_COVID)

In [ ]:
t2, p2 = T(Three_Day_Pre_COVID_Data,Three_Day_Post_COVID_Data)
t3, p3 = T(Five_Day_Pre_COVID_Data,Five_Day_Post_COVID_Data)
t4, p4 = T(Seven_Day_Pre_COVID_Data,Seven_Day_Post_COVID_Data)


t={'3 Day CAR':t2[0],'5 Day CAR':t3[0],'7 Day CAR':t4[0]}
p={'3 Day CAR':p2[0],'5 Day CAR':p3[0],'7 Day CAR':p4[0]}

COVID_t_test = pd.DataFrame.from_dict(t,orient='index',columns=['T-Stat'])
COVID_p_test = pd.DataFrame.from_dict(p,orient='index',columns=['P-Stat'])
COVID_Stats=COVID_t_test.merge(COVID_p_test,left_index=True, right_index=True)
COVID_Stats

In [ ]:
def plot_distribution(A1,A2,alpha):
  plt.figure()
  ax1=sns.distplot(A1.values)
  ax2=sns.distplot(A2.values)
  plt.axvline(np.mean(A1.values),color='b',linestyle='dashed',linewidth=5)
  plt.axvline(np.mean(A2.values),color='orange',linestyle='dashed',linewidth=5)

  stat, p=T(A1.values,A2.values)
  print('Statistics=%.3f, p=%.3f' % (stat,p))
  if(p > alpha):
    print('Same distributions (fail to reject H0)')
  else:
    print('Different distributions (reject H0)')
  
  return plt.figure

In [ ]:
plot_distribution(Three_Day_Pre_COVID_Data,Three_Day_Post_COVID_Data,0.05)

In [ ]:
plot_distribution(Three_Day_Pre_Demo_Data,Three_Day_Post_Demo_Data,0.05)

In [ ]:
plot_distribution(Five_Day_Pre_Demo_Data,Five_Day_Post_Demo_Data,0.05)

In [ ]:
plot_distribution(Five_Day_Pre_COVID_Data,Five_Day_Post_COVID_Data,0.05)

In [ ]:
plot_distribution(Seven_Day_Pre_Demo_Data,Seven_Day_Post_Demo_Data,0.05)

In [ ]:
plot_distribution(Seven_Day_Pre_COVID_Data,Seven_Day_Post_COVID_Data,0.05)